In [105]:
import numpy as np
import math

In [106]:
Xtrain_file = '../data/Xtrain.csv'
Ytrain_file = '../data/Ytrain.csv'

# load training files 
X_train = np.genfromtxt(Xtrain_file, delimiter=',').astype(int)
Y_train = np.genfromtxt(Ytrain_file, delimiter=',').astype(int)


# create 90-10 split for testing
X_test = X_train[-50:]
Y_test = Y_train[-50:]

X_train = X_train[:450]
Y_train = Y_train[:450]

### Training Voted Perceptron

In [246]:
Y_train[Y_train == 0] = -1
Y_train[Y_train == 1] = 1

Y_test[Y_test == 0] = -1
Y_test[Y_test == 1] = 1

In [322]:
# Init variables
c = np.zeros(1)
w = np.zeros((1, len(X_train[0])))

In [323]:
# PAPER IMPLEMENTATION
e = 10
y_hat = 0
t = 0
k = 0


for _ in range(e):
    for idx, (x, y) in enumerate(zip(X_train, Y_train)):
        y_hat = int(np.sign(np.dot(w[k], x)))
        
        if y_hat == y:
            c[k] += 1
        else:
            w = np.append(w, [w[k] + y*x], axis=0) 
            c = np.append(c, [1])
            k += 1
        

### Voted Perceptron Prediction 

In [324]:
# x_test = X_test[0]
y_hat = 0

y_pred = []


for x_test, l in zip(X_test, Y_test):
    for i in range(len(c)):
        y_hat += c[i] * int(np.sign(np.dot(w[i], x_test)))
    
    y_pred.append(int(np.sign(y_hat)))
    y_hat = 0

In [325]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_test, y_pred)

1.0

### Naive Bayes Training

In [ ]:
# train set size
train_size = float(len(Y_train))

# number of instances per class
class_0, class_1 = np.bincount(Y_train)

# log priors 
log_class_0_prior = math.log(class_0 / train_size)
log_class_1_prior = math.log(class_1 / train_size)

# total vocabulary
V = len(X_train[0])

# total tokens in each class (scalar)
class_0_wtot = 0 + V*0.5 # add one-smoothing
class_1_wtot = 0 + V*0.5 # add one-smoothing

# total per token in each class (vector)
class_0_vectot = np.full(len(X_train[0]), 0.5) # ones == add one-smoothing
class_1_vectot = np.full(len(X_train[0]), 0.5)

loglike_0_vec = np.zeros(len(X_train[0]), dtype=np.int)
loglike_1_vec = np.zeros(len(X_train[0]), dtype=np.int)

# sum of the total tokens
for x, y in zip(X_train, Y_train):        
    if y == 0:
        class_0_wtot += np.sum(x)
        class_0_vectot += x
    else: 
        class_1_wtot += np.sum(x)
        class_1_vectot += x

for i in range(V):
    loglike_0_vec[i] = math.log(class_0_vectot[i]/float(class_0_wtot))
    loglike_1_vec[i] = math.log(class_1_vectot[i]/float(class_1_wtot))

### Naive Bayes Prediction

In [ ]:
# prediction

sum_0 = log_class_0_prior
sum_1 = log_class_1_prior

for i, x in enumerate(X_test):
    for idx, val in enumerate(x):
        if val != 0:
            sum_0 += (loglike_0_vec[idx]*val)
            sum_1 += (loglike_1_vec[idx]*val)
        
        
    print(np.argmax([sum_0, sum_1]))
    print(Y_test[i])
    print("\n")
    sum_0 = log_class_0_prior
    sum_1 = log_class_1_prior